In [1]:
// Imports the Kotlin DataFrame library
%use dataframe

// Imports the Kotlin Kandy library
%use kandy

import kotlin.random.Random

// Simulation parameters
val years = 30
val simulations = 1000
val initialInvestment = 1.0

// Market probabilities
val bullProb = 0.75

// Portfolio returns
val portfolioA = mapOf("bull" to 1.20, "bear" to 0.80)
val portfolioB = mapOf("bull" to 1.25, "bear" to 0.65)

// Function to simulate one portfolio run and return the accumulated return for each year
fun simulatePortfolioAccumulatedReturns(returns: Map<String, Double>, rng: Random): List<Double> {
    var value = initialInvestment
    val accumulatedReturns = mutableListOf<Double>()

    repeat(years) {
        val isBull = rng.nextDouble() < bullProb
        val market = if (isBull) "bull" else "bear"
        value *= returns[market]!!

        // Calculate accumulated return for the current year
        val accumulatedReturn = (value - initialInvestment) / initialInvestment * 100
        accumulatedReturns.add(accumulatedReturn)
    }

    return accumulatedReturns
}

// Running simulations and storing accumulated returns for each year (for each portfolio)
val rng = Random(42)

val accumulatedResults = (1..simulations).map {
    val accumulatedReturnsA = simulatePortfolioAccumulatedReturns(portfolioA, rng)
    val accumulatedReturnsB = simulatePortfolioAccumulatedReturns(portfolioB, rng)
    mapOf("Simulation" to it, "PortfolioA" to accumulatedReturnsA, "PortfolioB" to accumulatedReturnsB)
}

// Transpose the results and calculate average accumulated return for each year
val accumulatedResultsList = accumulatedResults.map {
    it["PortfolioA"] as List<Double>
}
val accumulatedResultsBList = accumulatedResults.map {
    it["PortfolioB"] as List<Double>
}

// Transpose the results to calculate the average per year
val avgAccumulatedReturnA = (0 until years).map { year ->
    accumulatedResultsList.map { it[year] }.average()
}
val avgAccumulatedReturnB = (0 until years).map { year ->
    accumulatedResultsBList.map { it[year] }.average()
}

// Convert to DataFrame for plotting
val avgAccumulatedReturns = (1..years).map {
    mapOf(
        "Year" to it,
        "PortfolioA" to avgAccumulatedReturnA[it - 1],
        "PortfolioB" to avgAccumulatedReturnB[it - 1]
    )
}

data class YearlyAccumulatedReturn(val Year: Int, val PortfolioA: Double, val PortfolioB: Double)
val df = avgAccumulatedReturns.map {
    YearlyAccumulatedReturn(it["Year"] as Int, it["PortfolioA"] as Double, it["PortfolioB"] as Double)
}.toDataFrame()

// Plot the average accumulated return for both portfolios over 30 years
df.plot {
    line {
        x("Year") // X-axis: Year of the simulation
        y("PortfolioA") // Y-axis: accumulated return for Portfolio A
        color = Color.RED

    }
    line {
        x("Year") // X-axis: Year of the simulation
        y("PortfolioB") // Y-axis: accumulated return for Portfolio B
        color = Color.BLUE

    }

    layout {
        title = "Average Accumulated Return Over $years Years"
        xAxisLabel = "Year"
        yAxisLabel = "Accumulated Return (%)"
    }
}

// Count the number of simulations where Portfolio A outperforms Portfolio B and vice versa

var portfolioAOutperformsB = 0
var portfolioBOutperformsA = 0

accumulatedResults.forEach { result ->
    // Get the final accumulated return for each portfolio (last value in the list)
    val accumulatedA = result["PortfolioA"] as List<Double>
    val accumulatedB = result["PortfolioB"] as List<Double>

    // Compare the final accumulated returns (not just the last year's return)
    if (accumulatedA.last() > accumulatedB.last()) {
        portfolioAOutperformsB++
    } else {
        portfolioBOutperformsA++
    }
}

// Print the results
println("Number of simulations where Portfolio A outperforms Portfolio B: $portfolioAOutperformsB")
println("Number of simulations where Portfolio B outperforms Portfolio A: $portfolioBOutperformsA")



Number of simulations where Portfolio A outperforms Portfolio B: 644
Number of simulations where Portfolio B outperforms Portfolio A: 356
